In [1]:
import os
import sys
import base64
from pathlib import Path
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from bs4 import BeautifulSoup

In [2]:
path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/saideepbunny/Projects/Email-Assistant-using-Generative-AI


In [3]:
def get_email_body(payload):
    """Recursively extracts the best plain text body from the email payload."""
    if 'parts' in payload:
        for part in payload['parts']:
            body = get_email_body(part)
            if body and body != "(No content)":
                return body

    mime_type = payload.get('mimeType', '')
    data = payload.get('body', {}).get('data')

    if data:
        decoded = base64.urlsafe_b64decode(data).decode('utf-8', errors='ignore')

        if mime_type == "text/plain":
            return decoded.strip()
        elif mime_type == "text/html":
            soup = BeautifulSoup(decoded, "html.parser")
            return soup.get_text(separator=" ", strip=True)

    return "(No content)"


def extract_header(headers, name):
    """Helper to extract a specific header value from Gmail headers."""
    for header in headers:
        if header.get('name', '').lower() == name.lower():
            return header.get('value', '')
    return f"(No {name})"


def read_emails():
    creds = Credentials.from_authorized_user_file(f'{path}/config/token.json')
    service = build('gmail', 'v1', credentials=creds)

    results = service.users().messages().list(userId='me', maxResults=10).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No emails found.")
        return

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
        payload = msg_data.get('payload', {})
        headers = payload.get('headers', [])

        sender = extract_header(headers, 'from')
        subject = extract_header(headers, 'subject')
        date = extract_header(headers, 'date')

        body = get_email_body(payload)

        print("=" * 80)
        print(f"📬 From: {sender}")
        print(f"📅 Date: {date}")
        print(f"📝 Subject: {subject}")
        print(f"📨 Body:\n{body}")
        print("=" * 80 + "\n")


if __name__ == '__main__':
    read_emails()


📬 From: Amy at Adzuna <no-reply@adzuna.com>
📅 Date: Sun, 3 Aug 2025 15:11:39 +0000
📝 Subject: Methodist Hospital Specialty and Transplant is looking for Clinical ...
📨 Body:
Adzuna Job alert

617 new DATA OR SCIENTIST jobs in San Antonio, TX. View your latest job matches here:

https://www.adzuna.com/search?loc=185523&d=15&qor=Data+Scientist&adv=1&se=kmebJ3xw8BGhk6_tJW6AKw&nid=117074990&utm_source=alerts&utm_medium=email&utm_content=data_scientist_near_san_antonio_tx~san_antonio_tx~hostedapply~hostedapply~~~rec_false~0~10~617~sl1&utm_campaign=Alerts_01_08_25&utm_term=ab-mar4248-control

Edit your alerts here:

https://www.adzuna.com/my-alerts?id=117074990&vhash=8e019487159e2889488471df925b0137523c854a&utm_source=alerts&utm_medium=email&utm_content=data_scientist_near_san_antonio_tx~san_antonio_tx~hostedapply~hostedapply~~~rec_false~0~10~617~sl1&utm_campaign=Alerts_01_08_25&utm_term=ab-mar4248-control&autoedit=1

📬 From: Sebastian King from TopResume <sebastian.king@topresume.com>
📅 Dat